In [2]:
import sys
sys.path.append('../src')

import importlib
import numpy as np
from sklearn.covariance import empirical_covariance

import policies 

# Covariance Constrained Policy Testing

Testing the covariance constrained policy. Initially, this was giving a lot of violations. Initially thought that the problem might just not be feasible but after inspection it was. I switched the solver being used by cvxpy to Gurobi, which resolved the issue. 

The initial line in this, %%capture, redirects stdout from printing out the output. This is much nicer than redirecting the stdout manually, which doesn't work as anticipated inside of jupyter notebooks. Note: if your code is multithreaded, this no longer works for some reason. 

In [24]:
%%capture

importlib.reload(policies)

n = 50
n_features = 6
pred_dim = 6

np.random.seed(30)

# Gaussian xs, with true labels having a different linear relationship with xs in each coordinate
xs = np.random.normal(size=(n, n_features))
slopes = np.random.uniform(size = n_features)
ys = np.multiply(xs, slopes)

def meta_model(coord, slopes):
    def model(xs):
        preds = np.random.normal(size=(n, pred_dim))
        true_ys = np.multiply(xs, slopes)
        preds[:,coord] = true_ys[:,coord]
        return preds
    return model

model = meta_model(0, slopes)

alpha = 0.9
policy = policies.VarianceConstrained(pred_dim, model, 0.1, alpha, ys)
out = policy.run_given_preds(ys);

In [25]:
out

array([[3.45055097e-11, 2.99907272e-08, 4.77199150e-09, 9.99999948e-01,
        3.56356876e-09, 1.35077245e-08],
       [1.28539306e-09, 6.14705503e-02, 2.94406680e-07, 1.01343087e-08,
        9.38524038e-01, 5.10582094e-06],
       [5.58357438e-11, 7.56213593e-09, 3.61906979e-08, 9.99999937e-01,
        1.26381807e-09, 1.78635919e-08],
       [2.06121910e-11, 6.02228732e-08, 6.61801531e-09, 2.96357864e-10,
        1.81954442e-11, 9.99999933e-01],
       [3.57604655e-11, 1.68143012e-07, 6.12813941e-02, 4.88329329e-09,
        9.38718407e-01, 2.62891145e-08],
       [1.29919379e-11, 1.43779487e-09, 9.99999998e-01, 3.49842770e-11,
        1.21126657e-11, 9.97018835e-10],
       [5.92512938e-02, 6.55174358e-11, 6.68072187e-11, 9.54043725e-11,
        9.40748706e-01, 6.70293772e-11],
       [3.39062932e-07, 3.09978744e-08, 3.57435341e-08, 5.99839219e-02,
        9.40015635e-01, 3.71567871e-08],
       [1.27595341e-09, 5.19147844e-08, 1.16907101e-07, 5.99840042e-02,
        9.40015787e-01, 

Sanity check: 

1. Does each row sum to 1? Yes! Or at least, up to a tolerance, since there will be small floating point errors.

In [26]:
tolerance = 1e-3

print(np.sum(out, axis=1))
print("Violations: ", n-sum((np.isclose(np.sum(out, axis=1), np.ones(50), atol=tolerance))))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]
Violations:  0


2. Is each constraint bounded by 0 and 1?

In [27]:
print(f"Number of allocations greater than 1: {np.sum(out - tolerance > 1)}")
print(f"Number of allocations less than 0: {np.sum(out+tolerance < 0)}")

Number of allocations greater than 1: 0
Number of allocations less than 0: 0


3. Are the variance conditions being approx satisfied? I wrote yes here before but it seems like no?

In [28]:
cov = empirical_covariance(ys, assume_centered=False)

viol = np.zeros(len(out))
for i in range(len(out)):
    viol[i] = np.matmul(np.matmul(out[i], cov), np.transpose(out[i]))

print(f"Max violation: {alpha}")
print(f"Number of variance constraints which violate the max allowed variance: {sum(viol > alpha+0.1)}")

Max violation: 0.9
Number of variance constraints which violate the max allowed variance: 0


3. Is the covariance matrix actually measuring the correct thing? A sanity check that the diagonal of the matrix is equal to empirical variance.


In [29]:
print(f"Variances of each coordinate of the ys: \n {np.var(ys, axis=0)}")

print(f"Diagonal of the covariance matrix: \n {np.diagonal(cov)}")

Variances of each coordinate of the ys: 
 [0.82798883 0.00104122 0.00179195 0.10912913 1.02202326 0.00313415]
Diagonal of the covariance matrix: 
 [0.82798883 0.00104122 0.00179195 0.10912913 1.02202326 0.00313415]


4. Are all the constraints convex and are things feasible?

If C is positive semi-definite, then xCx^T <= val is a convex constraint on R^n. Maybe something went wrong in calculation of C? 

If the eigenvalues of C are positive, then C must be positive semi-definite. Here we see this is the case:

In [30]:
np.linalg.eigvals(cov) > 0

array([ True,  True,  True,  True,  True,  True])

If the only constraint was covariance, xCx^T will be bounded by the eigenvalues of covariance matrix, so this is a good spot check for how feasible things are

In [31]:
np.linalg.eigvals(cov)

array([1.04541420e+00, 8.10321387e-01, 1.04229047e-01, 2.73097313e-03,
       8.41962782e-04, 1.57097101e-03])